In [1]:
!pip install beautifulsoup4



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install selenium



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

# # 크롬 드라이버 생성
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# # Google 웹사이트에 접속
# driver.get("https://www.google.com")

# # 검색 입력창 찾기 (검색창의 이름이 'q')
# search_box = driver.find_element(By.NAME, 'q')

# # 검색어 입력
# search_box.send_keys('Jennie')

# # 검색 실행
# search_box.submit()
# time.sleep(5)

# # 결과 페이지 스크린샷 저장
# driver.save_screenshot('search_results.png')

# # 브라우저 종료
# driver.quit()

In [40]:
dtl_link.text in '제70류 유리와 유리제품 주: 1. 이 류에서 다음 각 목의 것은 제외한다. 가. 제3207호의 물품[예: 법랑ㆍ유약ㆍ유리프리트(frit)와 가루 모양ㆍ알갱이 모양ㆍ플레이크(flake) 모양인 그 밖의 유리] 나. 제71류의 물품(예: \xa0모조 신변장식용품) 다. 제8544호의 광섬유 케이블, 애자(제8546호...'


True

In [ ]:
import requests
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd


# 크롬 드라이버 생성
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))


# 페이지 열기
url = "https://unipass.customs.go.kr/clip/index.do"
driver.get(url)

# JavaScript가 실행될 시간을 기다리기
driver.implicitly_wait(10)  # 최대 10초 대기

# 페이지의 HTML 가져오기
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# 세계HS 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.CLASS_NAME,"M_ULS0200000000")
click_box.click()

# HS해설서 창을 찾아 click_box 변수에 저장 (By.CLASS_NAME 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="LEFTMENU_LNK_UI-ULS-0202-001Q"]/span')
click_box.click()

# 조회 창을 찾아 click_box 변수에 저장 (By.XPATH 방식)
click_box = driver.find_element(By.XPATH, '//*[@id="searchVo"]/div[2]/footer/button/span')
driver.execute_script("arguments[0].click();", click_box)


# Wait for the table to load
time.sleep(1)  # Adjust the wait time as needed

# Locate the table
table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')

# Parse the table HTML using BeautifulSoup
soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')

# Extract table rows
rows = soup.find_all('tr')

# Initialize an empty DataFrame
data = pd.DataFrame(columns=["구분", "년도", "단위", "HS분류", "내용", "Left Area", "Right Area"])

# # Iterate through the rows and extract data
# for row in rows:
#     cols = row.find_all('td')
#     cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
#     if cols:  # Check if the row has data
#         # Click on the `dtlInfo` link
#         dtl_link = row.find('a', class_='dtlInfo')
#         if dtl_link:
#             dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)
#             dtl_element.click()
#             time.sleep(2)  # Wait for the leftArea and rightArea to load

#             # Extract text from leftArea
#             left_area = driver.find_element(By.ID, 'divLft_tab4')
#             left_area_text = left_area.text.strip()

#             # Extract text from rightArea
#             right_area = driver.find_element(By.ID, 'divRght_tab4')
#             right_area_text = right_area.text.strip()

#             # Go back to the main page to continue scraping
#             # driver.back()
#             time.sleep(2)  # Wait for the page to load

#             # Add data to the DataFrame
#             data.loc[len(data)] = cols + [left_area_text, right_area_text]

# Loop through all 135 pages
for page in range(1, 136):
    print(f"Processing page {page}...")
    
    # Locate the table
    table = driver.find_element(By.XPATH, '//*[@id="ULS0202001Q_T1_table1"]/tbody')
    
    # Parse the table HTML using BeautifulSoup
    soup = BeautifulSoup(table.get_attribute("outerHTML"), 'html.parser')
    
    # Extract table rows
    rows = soup.find_all('tr')
    
    # Iterate through the rows and extract data
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
        if cols:  # Check if the row has data
            # Click on the `dtlInfo` link
            dtl_link = row.find('a', class_='dtlInfo')
            if dtl_link:
                # Re-locate the element dynamically using its text or another unique attribute
                if dtl_link.text in '류':
                    driver.find_element(By.XPATH, f"//a[text()='{dtl_link.text.strip()}']")
                elif dtl_link.text in '제76류 알루미늄과 그 제품 소호주: 1. 이 류에서 다음 각 목의 용어는 아래에서 정하는 바에 따른다. 가. 합금하지 않은 알루미늄 알루미늄의 함유량이 전 중량의 100분의 99 이상인 금속으로서 그 밖의 원소의 함유량이 중량비로 다음 표에 열거한 한도를 초과하지 않아야 한다. 그 밖의 원소표  나. 알루미...':
                    driver.find_element(By.XPATH, f"//a[text()='{dtl_link.text.strip()}']")
                else: 
                    dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)
                driver.execute_script("arguments[0].click();", dtl_element)  # Use JavaScript click to avoid staleness
                time.sleep(0.2)  # Wait for the leftArea and rightArea to load

                # Extract text from leftArea
                left_area = driver.find_element(By.ID, 'divLft_tab4')
                left_area_text = left_area.text.strip()

                # Extract text from rightArea
                right_area = driver.find_element(By.ID, 'divRght_tab4')
                right_area_text = right_area.text.strip()

                # Add data to the DataFrame
                data.loc[len(data)] = cols + [left_area_text, right_area_text]
    
    # Navigate to the next page
    try:
        next_page = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[@href="#{page + 1}"]')
        next_page.click()
        time.sleep(0.2)  # Wait for the page to load
    except Exception as e:
        print(f"Could not navigate to page {page + 1}: {e}")
        break

# Display the DataFrame

# Save the data to a CSV file
data.to_csv('HSdescription_data.csv', index=False, encoding='utf-8-sig')

Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44...
Processing page 45...
Processing page 46.

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"제70류 유리와 유리제품 주: 1. 이 류에서 다음 각 목의 것은 제외한다. 가. 제3207호의 물품[예: 법랑ㆍ유약ㆍ유리프리트(frit)와 가루 모양ㆍ알갱이 모양ㆍ플레이크(flake) 모양인 그 밖의 유리] 나. 제71류의 물품(예:  모조 신변장식용품) 다. 제8544호의 광섬유 케이블, 애자(제8546호..."}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00553433+25059]
	(No symbol) [0x004DCE34]
	(No symbol) [0x003BBEC3]
	(No symbol) [0x003FFD86]
	(No symbol) [0x003FFFCB]
	(No symbol) [0x0043D952]
	(No symbol) [0x00421F44]
	(No symbol) [0x0043B51E]
	(No symbol) [0x00421C96]
	(No symbol) [0x003F3FAC]
	(No symbol) [0x003F4F3D]
	GetHandleVerifier [0x00845593+3113795]
	GetHandleVerifier [0x0085A25A+3198986]
	GetHandleVerifier [0x00852A32+3168226]
	GetHandleVerifier [0x005F32A0+680016]
	(No symbol) [0x004E577D]
	(No symbol) [0x004E2A28]
	(No

In [43]:
for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]  # Extract text and clean up whitespace
        if cols:  # Check if the row has data
            # Click on the `dtlInfo` link
            dtl_link = row.find('a', class_='dtlInfo')
            if dtl_link:
                # Re-locate the element dynamically using its text or another unique attribute
                if dtl_link.text in "제76류 알루미늄과 그 제품 소호주: 1. 이 류에서 다음 각 목의 용어는 아래에서 정하는 바에 따른다. 가. 합금하지 않은 알루미늄 알루미늄의 함유량이 전 중량의 100분의 99 이상인 금속으로서 그 밖의 원소의 함유량이 중량비로 다음 표에 열거한 한도를 초과하지 않아야 한다. 그 밖의 원소표  나. 알루미...":
                    driver.find_element(By.XPATH, f"//a[text()='{dtl_link.text.strip()}']")
                else: 
                    dtl_element = driver.find_element(By.LINK_TEXT, dtl_link.text)
                driver.execute_script("arguments[0].click();", dtl_element)  # Use JavaScript click to avoid staleness
                time.sleep(0.1)  # Wait for the leftArea and rightArea to load

                # Extract text from leftArea
                left_area = driver.find_element(By.ID, 'divLft_tab4')
                left_area_text = left_area.text.strip()

                # Extract text from rightArea
                right_area = driver.find_element(By.ID, 'divRght_tab4')
                right_area_text = right_area.text.strip()

In [44]:
left_area_text

'76.03 - 알루미늄의 가루와 플레이크(flake)\n7603.10 - 비(非)층상조직인 가루\n7603.20 - 층상조직인 가루와 플레이크(flake)\n이 호에는 제15부의 주 제8호나목에 규정한 알루미늄 가루와 플레이크(flake)를 분류한다. 일반적으로 이들 물품은 구리의 가루에 상응하는 것이며 따라서 제7406호의 해설규정이 이 호에 준용된다. 그러나 알루미늄의 가루와 플레이크(flake)는 발광탄의 열 발생원[예: 테르밋(thermit) 공정에서]으로 사용하거나 부식으로부터 다른 금속을 보호하기 위하여[예: 캘러라이징(calorising)ㆍ금속접합]ㆍ로켓(rocket) 추진체와 특수 시멘트 제조용에 사용한다.\n이 호에는 다음의 것을 제외한다.\n(a) 착색제ㆍ페인트나 그 밖의 이와 유사한 물품으로 조제된 가루나 플레이크[예: 다른 착색제와 혼합하여 제조한 것, 결합제(binder)나 용제(solvent)를 이용해서 현탁액(suspension)ㆍ분산물(dispersion)이나 페이스트(paste) 상태로 조제된 것](제32류)\n(b) 알루미늄 펠릿(pellet)(제7601호)\n(c) 알루미늄박(箔)을 절단한 스팽글(spangle)(제8308호)'

In [34]:
data

,구분,년도,단위,HS분류,내용,Left Area,Right Area
0,WCO,2022,류,01,제1류 살아 있는 동물 주: 1. 이 류에는 다음 각 목의 것을 제외한 모든 살아 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,WCO,2022,부,01,제 1 부 살아 있는 동물과 동물성 생산품 주: 1. 이 부에 열거된 동물의 특정 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
2,WCO,2022,호,0101,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+) - 말 0101.21 -- 번식...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
3,WCO,2022,호,0102,01.02 - 살아 있는 소(+) - 축우(畜牛) 0102.21 -- 번식용 010...,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
4,WCO,2022,호,0103,01.03 - 살아 있는 돼지(+) 0103.10 - 번식용 - 기타 0103.91...,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...
5,WCO,2022,호,0104,01.04 - 살아 있는 면양과 염소 0104.10 - 면양 0104.20 - 염소...,01.04 - 살아 있는 면양과 염소\n0104.10 - 면양\n0104.20 - ...,01.04 - Live sheep and goats.\n0104.10 - Sheep...
6,WCO,2022,호,0105,01.05 - 살아 있는 가금(家禽)류[닭(갈루스 도메스티쿠스(Gallus dome...,01.05 - 살아 있는 가금(家禽)류[닭(갈루스 도메스티쿠스(Gallus dome...,"01.05 - Live poultry, that is to say, fowls of..."
7,WCO,2022,호,0106,01.06 - 그 밖의 살아 있는 동물 - 포유동물 0106.11 -- 영장류 01...,01.06 - 그 밖의 살아 있는 동물\n- 포유동물\n0106.11 - 영장류\n...,01.06 - Other live animals.\n- Mammals :\n0106...
8,WCO,2022,류,02,제2류 육과 식용 설육(屑肉) 주: 1. 이 류에서 다음 각 목의 것은 제외한다. ...,02.01 - 쇠고기(신선한 것이나 냉장한 것으로 한정한다)\n0201.10 - 도...,"02.01 Meat of bovine animals, fresh or chilled..."
9,WCO,2022,부,02,제 2 부 식물성 생산품 주: 1. 이 부에서 “펠릿(pellet)”이란 직접 압축...,06.01 - 인경(鱗莖)ㆍ괴경(塊莖)ㆍ괴근(塊根)ㆍ구경(球莖)ㆍ관근(冠根)ㆍ근경(...,"06.01 Bulbs, tubers, tuberous roots, corms, cr..."


In [41]:
driver.find_element(By.XPATH, f"//a[text()='{dtl_link.text.strip()}']")

<selenium.webdriver.remote.webelement.WebElement (session="bf4686d27fe8dfa3766ec5d9bdbc4ec1", element="f.78EA1EC545E59180FA465B48CB075FFE.d.CD39D555F2A319EE7247F71599A84E0C.e.1328133")>

In [ ]:
driver.find_element(By.XPATH, dtl_link.text)
driver.find_element(By.LINK_TEXT, dtl_link.text)

InvalidSelectorException: Message: invalid selector
from javascript error: {"status":32,"value":"Unable to locate an element with the xpath expression 09.03 - 마테(maté) 마테(maté)는 남미에서 재배되는 서양감탕나무과(holly family)에 속하는 어떤 관목의 잎을 말린 것으로서, 때로는 “파라과이차(Paraguay tea)”나 “제수이트 티(Jesuits' tea)”로 알려져 있다. 마테는 극소량의 카페인을 함유하는 음료를 조제하는데 넣어... because of the following error:\nSyntaxError: Failed to execute 'evaluate' on 'Document': The string '09.03 - 마테(maté) 마테(maté)는 남미에서 재배되는 서양감탕나무과(holly family)에 속하는 어떤 관목의 잎을 말린 것으로서, 때로는 “파라과이차(Paraguay tea)”나 “제수이트 티(Jesuits' tea)”로 알려져 있다. 마테는 극소량의 카페인을 함유하는 음료를 조제하는데 넣어...' is not a valid XPath expression."}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
	GetHandleVerifier [0x00553433+25059]
	(No symbol) [0x004DCE34]
	(No symbol) [0x003BBEC3]
	(No symbol) [0x003C19D1]
	(No symbol) [0x003C38EA]
	(No symbol) [0x

In [27]:
# Re-locate the element dynamically using its text or another unique attribute
dtl_element2 = driver.find_element(By.LINK_TEXT, dtl_link.text)
driver.execute_script("arguments[0].click();", dtl_element2)  # Use JavaScript click to avoid staleness

In [16]:
dtl_link

<a class="dtlInfo" href="#">09.03 - 마테(maté) 마테(maté)는 남미에서 재배되는 서양감탕나무과(holly family)에 속하는 어떤 관목의 잎을 말린 것으로서, 때로는 “파라과이차(Paraguay tea)”나 “제수이트 티(Jesuits' tea)”로 알려져 있다. 마테는 극소량의 카페인을 함유하는 음료를 조제하는데 넣어...</a>

In [17]:
data

,구분,년도,단위,HS분류,내용,Left Area,Right Area
0,WCO,2022,류,01,제1류 살아 있는 동물 주: 1. 이 류에는 다음 각 목의 것을 제외한 모든 살아 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,WCO,2022,부,01,제 1 부 살아 있는 동물과 동물성 생산품 주: 1. 이 부에 열거된 동물의 특정 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
2,WCO,2022,호,0101,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+) - 말 0101.21 -- 번식...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
3,WCO,2022,호,0102,01.02 - 살아 있는 소(+) - 축우(畜牛) 0102.21 -- 번식용 010...,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
4,WCO,2022,호,0103,01.03 - 살아 있는 돼지(+) 0103.10 - 번식용 - 기타 0103.91...,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...
...,...,...,...,...,...,...,...
91,WCO,2022,호,0814,"08.14 - 감귤류의 껍질과 멜론(수박을 포함한다)의 껍질(신선한 것, 냉동하거나...","08.14 - 감귤류의 껍질과 멜론(수박을 포함한다)의 껍질(신선한 것, 냉동하거나...",08.14 Peel of citrus fruit or melons (includin...
92,WCO,2022,류,09,제9류 커피ㆍ차ㆍ마테(maté)ㆍ향신료 주: 1. 제0904호부터 제0910호까지의...,"09.01 - 커피(볶았는지, 카페인을 제거했는지에 상관없다), 커피의 껍데기와 껍...","09.01 Coffee, whether or not roasted or decaff..."
93,WCO,2022,부,09,"제 9 부 목재와 그 제품, 목탄, 코르크와 그 제품, 짚․에스파르토(esparto...","44.01 - 땔나무(통나무, 목편, 작은 가지, 다발이나 이와 유사한 모양으로 한...","44.01 - Fuel wood, in logs, in billets, in twi..."
94,WCO,2022,호,0901,"09.01 - 커피(볶았는지, 카페인을 제거했는지에 상관없다), 커피의 껍데기와 껍...","09.01 - 커피(볶았는지, 카페인을 제거했는지에 상관없다), 커피의 껍데기와 껍...","09.01 Coffee, whether or not roasted or decaff..."


In [2]:
data

,구분,년도,단위,HS분류,내용,Left Area,Right Area
0,WCO,2022,류,01,제1류 살아 있는 동물 주: 1. 이 류에는 다음 각 목의 것을 제외한 모든 살아 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,WCO,2022,부,01,제 1 부 살아 있는 동물과 동물성 생산품 주: 1. 이 부에 열거된 동물의 특정 ...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
2,WCO,2022,호,0101,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+) - 말 0101.21 -- 번식...,01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
3,WCO,2022,호,0102,01.02 - 살아 있는 소(+) - 축우(畜牛) 0102.21 -- 번식용 010...,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
4,WCO,2022,호,0103,01.03 - 살아 있는 돼지(+) 0103.10 - 번식용 - 기타 0103.91...,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...
...,...,...,...,...,...,...,...
858,WCO,2022,호,6910,69.10 - 도자제의 설거지통ㆍ세면대ㆍ세면대용 받침ㆍ목욕통ㆍ비데ㆍ수세식 변기통ㆍ수...,69.10 - 도자제의 설거지통ㆍ세면대ㆍ세면대용 받침ㆍ목욕통ㆍ비데ㆍ수세식 변기통ㆍ수...,"69.10 - Ceramic sinks, wash basins, wash basin..."
859,WCO,2022,호,6911,69.11 - 자기제의 식탁용품ㆍ주방용품ㆍ그 밖의 가정용품ㆍ화장용품 6911.10 ...,69.11 - 자기제의 식탁용품ㆍ주방용품ㆍ그 밖의 가정용품ㆍ화장용품\n6911.10...,"69.11 - Tableware, kitchenware, other househol..."
860,WCO,2022,호,6912,69.12 - 도자제의 식탁용품ㆍ주방용품ㆍ그 밖의 가정용품ㆍ화장용품(자기제의 것은 ...,69.12 - 도자제의 식탁용품ㆍ주방용품ㆍ그 밖의 가정용품ㆍ화장용품(자기제의 것은 ...,"69.12 - Ceramic tableware, kitchenware, other ..."
861,WCO,2022,호,6913,69.13 - 도자제의 작은 조각상과 그 밖의 장식용 제품 6913.10 - 자기제...,69.13 - 도자제의 작은 조각상과 그 밖의 장식용 제품\n6913.10 - 자기...,69.13 - Statuettes and other ornamental cerami...
